In [ ]:
!pip install openai tiktoken chromadb langchain

In [1]:
import dotenv
import os

from genai.credentials import Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.model import Model

# from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
# from ibm_watson_machine_learning.foundation_models import Model
# from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
# from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [2]:
dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=15,
    stop_sequences=['\n\n']
).dict()

# api_key = os.getenv("API_KEY", None)
# project_id = os.getenv("PROJECT_ID", None)

# creds = {
#     "url"    : "https://us-south.ml.cloud.ibm.com",
#     "apikey" : api_key
# }

# params = {
#     GenParams.DECODING_METHOD:"greedy",
#     GenParams.MAX_NEW_TOKENS:300,
#     GenParams.MIN_NEW_TOKENS:15,
#     GenParams.REPETITION_PENALTY:2
# }

In [3]:
# model = Model(ModelTypes.LLAMA_2_70B_CHAT,creds,params,project_id)
# model = Model(model="bigcode/starcoder",credentials=creds, params=params)

llm = LangChainInterface(model="bigcode/starcoder", credentials=creds, params=params)

In [4]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [5]:
loader = GenericLoader.from_filesystem(
    "../../samples/",
    glob="**/*",
    suffixes=[".sql"],
    parser=LanguageParser(),
    # parser=LanguageParser(language=Language.SQL, parser_threshold=500)
)
documents = loader.load()
len(documents)

2

In [7]:
from sql_metadata import Parser

texts = []
for doc in documents:
    parser = Parser(doc.page_content)
    for key in parser.subqueries:
        print(f"subquery- {key} - {parser.subqueries[key]}")
        texts += parser.subqueries[key]

# len(texts)
# for text in texts:
#     print(text)

Not supported query type: /** RAQ **/							  
								  
sel dtab1.*,                                                 
case when PI_Warning = 'Y' then 'Y'                                                 
         when PI_Warning is null then null else 'N' end  "Professional_Investor",                                        
case when Vulnerable_Customer = 'Y' then 'Y'                                                 
        when Vulnerable_Customer is null then null else 'N' end "Vulnerable_Customer",                                        
case when With_BEA_Invest_Acct_Rels = 'Y' then 'Y' else 'N' end "With_BEA_Invest_Acct_Rels"                                               
from                                                
                                                
(sel     e.party_intl_nbr,                                            
        q.doc_type_Code,                                                           
        q.issue_country_code,                           

ValueError: Not supported query type!

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings
db = Chroma.from_documents(texts, HuggingFaceInstructEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "Generate a SQL to create Table with 2 column: Name, Id"
result = qa(question)
result['answer']

In [ ]:
questions = [
    "What is the class hierarchy?",
    "What classes are derived from the Chain class?",
    "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")